### 실습 개요
- <작성중>

### 사전준비
- 이전 과정을 함수로 만듬

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import functions as fs
from numpy import dot
from numpy.linalg import norm

def process_file(file_path):
    # 비디오 파일 -> ndarray 변환
    video_array, tot_duration, tot_frames = fs.video_2_ndarray(file_path)

    # 프레임 간 유사도 계산
    similarity_list = []
    for frame_1, frame_2 in zip(video_array[:-1], video_array[1:]):
        vector_1 = frame_1.reshape(-1) / 255.0
        vector_2 = frame_2.reshape(-1) / 255.0
        similarity = cos_sim(vector_1, vector_2)
        similarity_list.append(similarity)

    return {
        "similarity_list": similarity_list,
        "tot_duration": tot_duration,
        "tot_frames": tot_frames
    }
    
def process_dict(data, lower_sim_threshold=0.5):
    
    def cos_sim(A, B):
        return dot(A, B) / (norm(A) * norm(B))

    similarity_list = data["similarity_list"]
    tot_duration = data["tot_duration"]
    tot_frames = data["tot_frames"]

    # 유사도 DataFrame 생성
    df = pd.DataFrame(similarity_list, columns=['similarity'])
    
    # 유사도가 임계치 이하면 True(1) 아니면 False(0)
    df['lower_sim'] = (df['similarity'] < lower_sim_threshold)
    
    # lower_sim의 누적합(Cumulative Sum)을 구함
    df['cumsum'] = df['lower_sim'].cumsum()
    
    # cumsum값을 기준으로 grouping
    df_takes = df.groupby(df['cumsum'])
    
    # Group별 frame개수, 시작시간, 재생시간, 활동량 계산
    frame_count = []
    start = []
    duration = []
    activity = []

    for group_number, group_df in df_takes:

        min_frame_number = group_df.index.min() + 1
        max_frame_number = group_df.index.max() + 1
        v_frame_count = max_frame_number - min_frame_number + 1
        v_start = tot_duration * (min_frame_number / tot_frames)
        v_duration = tot_duration * (v_frame_count / tot_frames)

        refined_df = group_df[group_df['similarity'] > lower_sim_threshold]
        similarity_mean = refined_df['similarity'].mean()
        similarity_mean_power = similarity_mean**100
        v_activity_intensity = 1 - similarity_mean_power
        
        frame_count.append(v_frame_count)
        start.append(v_start)
        duration.append(v_duration)
        activity.append(v_activity_intensity)

    # Group DataFrame 생성
    group_df = pd.DataFrame({
        'frame_count': frame_count,
        'start': start,
        'duration': duration,
        'activity': activity
    })
    
    # HTML로 변환하여 반환
    return group_df

#### 1. 영상 분석 및 웹 대시보드 구현

In [ ]:
def process_video(file):

    data = process_file(file.name)
    group_df = process_dict(data, lower_sim_threshold=0.9)
    return group_df.to_html(), file.name

# Gradio 인터페이스
iface = gr.Interface(
    fn=process_video,
    inputs=gr.File(label="동영상 파일을 업로드하세요"),
    outputs=[
        gr.HTML(label="유사도 분석 결과"),   # 첫 번째 출력: HTML
        gr.Video(label="동영상 재생")      # 두 번째 출력: Video
    ],
    title="비디오 분석 & 유사도 결과",
    description="업로드한 동영상의 연속 프레임 간 유사도(코사인 유사도)를 계산하여, 통계치 및 액티비티를 표로 보여줍니다."
)

# 실행
iface.launch()

#### 2. 파이프라인 아키텍처를 통한 영상 분석 및 웹 대시보드 구현
- `VideoSimilarityExtractor` : 업로드한 비디오로부터 프레임 추출 & 연속 프레임 간 유사도 계산
- `ActivityDataAggregator` : 유사도 데이터 기반 그룹별 통계 및 활동량 계산
- `VideoActivityInterface` : Gradio로 웹 UI 제공, 비디오 재생 & 분석 결과 시각화
- `Pipeline` : 여러 개의 단계(클래스)를 순차적으로 연결해주는 역할. |(OR) 연산자로 체이닝을 하면, 각 클래스를 리스트로 묶어 최종적으로 한 번의 호출(`executer()`)만으로 앞에서부터 순차적으로 처리 과정을 진행하도록 만듬.

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import functions as fs  # video_2_ndarray가 들어있는 사용자 정의 모듈
from numpy import dot
from numpy.linalg import norm

class VideoSimilarityExtractor:
    """
    1) 비디오 파일을 입력받아:
       - np.ndarray 형태로 변환
       - 연속되는 프레임 간 유사도 리스트 계산
       - 비디오 전체 시간(tot_duration), 전체 프레임(tot_frames) 구하기
       
    output : similarity_list, tot_duration, tot_frames 를 담은 dictionary
    """

    def extract_video_similarity(self, file):
        return process_file(file.name)

    def __or__(self, other):
        """ 
        파이프라인 체이닝을 위한 OR 연산자 오버로딩 
        예) VideoSimilarityExtractor() | ActivityDataAggregator() 
        """
        return Pipeline([self, other])


class ActivityDataAggregator:
    """
    2) (similarity_list, tot_duration, tot_frames) dict를 받아서:
       - similarity_list를 DataFrame으로 구성
       - 임계치 기반으로 Grouping & 통계량 계산
       - 최종 activity가 포함된 DataFrame 생성하여 반환
    """
    def __init__(self, lower_sim_threshold):
        self.lower_sim_threshold = lower_sim_threshold

    def aggregate_activity_data(self, data):        
        return process_dict(data, lower_sim_threshold=self.lower_sim_threshold)

    def __or__(self, other):
        """ 파이프라인 체이닝을 위한 OR 연산자 오버로딩 """
        return Pipeline([self, other])


class Pipeline:
    """
    연쇄된 객체들을 순차적으로 연결해주는 파이프라인 클래스
    ex) VideoSimilarityExtractor() | ActivityDataAggregator() | VideoActivityInterface()
    """

    def __init__(self, steps):
        self.steps = steps

    def __or__(self, other):
        self.steps.append(other)
        return self

    def __call__(self):
        """
        executer() 로 호출했을 때 가장 마지막 스텝(예: VideoActivityInterface)에게
        앞선 모든 스텝 리스트를 넘겨주고 실행 (launch) 되도록 설정
        """
        final_step = self.steps[-1]
        if hasattr(final_step, "set_pipeline"):
            final_step.set_pipeline(self.steps[:-1])
        return final_step()


class VideoActivityInterface:
    """
    3) Gradio 기반으로:
       - 비디오 파일 업로드 시 첫 번째 클래스(VideoSimilarityExtractor)에서 유사도 데이터를 구하고
       - 두 번째 클래스(ActivityDataAggregator)에서 Group DataFrame을 만든 뒤
       - DataFrame.to_html() 결과를 화면에 표시
    """

    def set_pipeline(self, pipeline_steps):
        """
        앞선 스텝들(VideoSimilarityExtractor, ActivityDataAggregator) 인스턴스를 받아 저장
        """
        self.pipeline = pipeline_steps

    def __call__(self):
        """
        executer()로 실행되면 Gradio 인터페이스가 열림
        """

        def process_video(file):
            # 첫 번째 스텝 → 두 번째 스텝 순으로 처리 후 HTML로 반환
            data_dict = self.pipeline[0].extract_video_similarity(file)     # VideoSimilarityExtractor
            df_result = self.pipeline[1].aggregate_activity_data(data_dict)  # ActivityDataAggregator
            return df_result.to_html(), file.name

        # Gradio 인터페이스 설정
        iface = gr.Interface(
            fn=process_video,
            inputs=gr.File(label="동영상 파일을 업로드하세요"),
            outputs=[
                gr.HTML(label="유사도 분석 결과"),   # 첫 번째 출력: HTML
                gr.Video(label="동영상 재생")      # 두 번째 출력: Video
            ],
            title="비디오 분석 & 유사도 결과",
            description="업로드한 동영상의 연속 프레임 간 유사도(코사인 유사도)를 계산하여, 통계치 및 액티비티를 표로 보여줍니다."

        )

        iface.launch()

In [ ]:
# 파이프라인 생성
vs_extractor  = VideoSimilarityExtractor()
ad_aggregator = ActivityDataAggregator(lower_sim_threshold=0.9)
va_interface  = VideoActivityInterface()

executer = vs_extractor | ad_aggregator | va_interface 

# 실행
executer()

### 생각 해보기
파이프라인 아키텍쳐를 사용할 때 장단점은 무엇인가?